# Reviews de Amazon basado en NLP - Proyecto Final
#### **Objetivo: Predicción de ratings basado en reviews de videojuegos vendidos Amazon**

En la plataforma de Amazon, los usuarios pueden ingresar sus comentarios y ratings con respecto a un producto adquirido. Sin embargo, muchos usuarios solo colocan un review, lo cual dificulta a Amazon mantener un registro de calificaciones por cada uno de sus registros. Por ello, este modelo busca predecir la clasificación dada a raíz de un comentario mediante un sistema de rating del 1 al 5.

- Input: El videojuego es increíble. Garantiza muchas horas de diversión!
- Output: 5

# Importar librerias

In [ ]:
import pandas as pd
import numpy as np
## Librerias para graficación
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el Dataset

## Lectura del dataset

In [ ]:
!wget "https://huggingface.co/api/datasets/amazon_us_reviews/parquet/Video_Games_v1_00/train/0.parquet"
!wget "https://huggingface.co/api/datasets/amazon_us_reviews/parquet/Video_Games_v1_00/train/1.parquet"
!wget "https://huggingface.co/api/datasets/amazon_us_reviews/parquet/Video_Games_v1_00/train/2.parquet"

In [ ]:
# Se leen todos los datasets y se unen en uno solo
df0 = pd.read_parquet("0.parquet")
df1 = pd.read_parquet("1.parquet")
df2 = pd.read_parquet("2.parquet")
df = pd.concat([df0, df1, df2], axis=0)

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
sns.catplot(x='star_rating', kind='count', color='r', data=df)
plt.title('Distribución de Rating')
plt.xlabel('star_rating')
plt.ylabel('Conteo')

## Sample del dataset

In [ ]:
# Funcion para balancear el dataset
def balancear_dataset_por_rating(data_frame):
    # Obtenemos el recuento de cada valor de "rating"
    conteo_ratings = data_frame['star_rating'].value_counts()

    # Obtenemos el mínimo de ocurrencias de "rating" que queremos mantener
    min_ocurrencias = min(conteo_ratings)

    # Filtramos las filas que tienen "rating" mayor o igual al mínimo especificado
    data_frame_balanceado = data_frame.groupby('star_rating').apply(lambda x: x.sample(min_ocurrencias, random_state=42))

    # Restablecemos el índice del DataFrame resultante y eliminamos el índice antiguo
    data_frame_balanceado.reset_index(drop=True, inplace=True)

    return data_frame_balanceado

In [ ]:
df_train = balancear_dataset_por_rating(df)

In [ ]:
df_train.shape

In [ ]:
# Creamos una nueva columna con el titulo y la review del juego
df_train['text'] = df_train['review_headline'] + ' ' + df_train['review_body']

In [ ]:
# Se obtiene una muestra del dataset
df_train = df_train.sample(200000,random_state=42)
df_train.reset_index(drop=True, inplace=True)
df_train = df_train[["text", "star_rating"]]
df_train.head(5)

In [ ]:
sns.catplot(x='star_rating', kind='count', color='r', data=df_train)
plt.title('Distribución de Rating')
plt.xlabel('star_rating')
plt.ylabel('Conteo')

# Modelamiento por Arquitecturas

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
stemmer = nltk.stem.SnowballStemmer('english')
nltk.download('stopwords')

## BOW

In [ ]:
def processing_text(texto):
    # Paso 1: Remover con un expresión regular carateres especiales (no palabras).
    processed_feature = re.sub(r'\W', ' ', str(texto))
    # Paso 2: Remover ocurrencias de caracteres individuales
    processed_feature= re.sub(r'\s+[a-zA-Z]\s+', ' ', processed_feature)
    processed_feature = re.sub(r'\^[a-zA-Z]\s+', ' ', processed_feature)
    # Paso 3: Remover números (Ocurrencias muy esporádicas en nuestro dataset)
    processed_feature = re.sub(r'[0-9]+', ' ', processed_feature)
    # Paso 4: Simplificar espacios concecutivos a un único espacio entre palabras
    processed_feature = re.sub(' +', ' ', processed_feature)
    # Paso 5: Pasar todo el texto a minúsculas
    processed_feature = processed_feature.lower()
    # Paso 6: Aplicar stemming. Es una forma de enviar las palabras a una raiz común simplificando de esta manera el vocabulario.
    #         por ejemplo las palabras (absurdo, absurdos) que estan en el review 2895 seran llevados a la raiz común "absurd"
    #         y de esta forma se evita tener dos palabras diferentes con el mismo significado en nuestro vocabulario.
    processed_feature = " ".join([stemmer.stem(i) for i in processed_feature.split()])

    return processed_feature

In [ ]:

# texto_para_procesar y labels respectivamente
texto_para_procesar = df_train['text'].values
labels = df_train['star_rating'].values

# El texto ya procesado de cada ejemplo en nuestro dataset lo almacenaremos en la variable "texto_procesado"
texto_procesado = []
for sentence in range(0, len(texto_para_procesar)):
    procesado = processing_text(texto_para_procesar[sentence])
    texto_procesado.append(procesado)

## Naives Bayes (MultinomialNB)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay

In [ ]:
def create_model_naives(texto_features):
    # Partición del dataset: Seleccionar 80% para entrenamiento, 20% pruebas.
    X_train, X_test, y_train, y_test = train_test_split(texto_features, labels, test_size=0.2, random_state=0)

    # Modelo: Naive Bayes
    model = MultinomialNB()
    model.fit(X_train, y_train)

    # Reporte de clasificación
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions, digits=4))

    # Matriz de confusión
    cm = confusion_matrix(y_test, predictions,labels=model.classes_)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=model.classes_)
    disp.plot()

    return model

### Representacion 1

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=1000, stop_words=stopwords.words('english'))

# Construimos el vocabulario y tambien transformamos el texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_naives(texto_features)

### Representacion 2


In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=2500, stop_words=stopwords.words('english'))

# Construimos el vocabulario y tambien transformamos el texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_naives(texto_features)

### Representacion 3


In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords.words('english'))

# Construimos el vocabulario y tambien transformamos el texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_naives(texto_features)

## Logistic Regresion (SDGClassfier)

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
def create_model_sdg(texto_features, eta = 0.1):
    # Partición del dataset: Seleccionar 80% para entrenamiento, 20% pruebas.
    X_train, X_test, y_train, y_test = train_test_split(
        texto_features, labels, test_size=0.2, random_state=0)

    # Modelo: SGDClassifier
    model = SGDClassifier(loss='log_loss', learning_rate='constant', eta0=eta)
    model.fit(X_train, y_train)

    # Reporte de clasificación
    predictions = model.predict(X_test)
    print(classification_report(y_test, predictions, digits=4))

    # Matriz de confusión
    cm = confusion_matrix(y_test, predictions, labels=model.classes_)
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, display_labels=model.classes_)
    disp.plot()

    return model

### Representacion 1

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords.words('english'))

# Construimos el vocabulario y tambien transformamos el texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_sdg(texto_features,eta=100)

### Representacion 2

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords.words('english'))

# Construimos el vocabulario y tambien transformamos el texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_sdg(texto_features,eta=10)

### Representacion 3

In [ ]:
# Bolsa de palabras
vectorizer = CountVectorizer(max_features=3000, stop_words=stopwords.words('english'))

# Construimos el vocabulario y tambien transformamos el texto
texto_features = vectorizer.fit_transform(texto_procesado).toarray().astype("float16")

# Creamos el modelo y sus metricas
create_model_sdg(texto_features,eta=0.0001)

## Recurrent Neural Network (LSTM - Long short-term memory)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

In [ ]:
X = df_train['text'].values
y = df_train['star_rating'].values

In [ ]:
# Transformar las etiquetas categóricas en valores numéricos
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_categorical = to_categorical(y_encoded)

### Representacion 1

In [ ]:
max_words = 10000
max_sequence_length = 100

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_categorical, test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 64
epochs = 50
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss en el conjunto de prueba: {loss}, Accuracy en el conjunto de prueba: {accuracy}')

In [ ]:
# Reporte de Clasificación
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

class_labels = label_encoder.classes_.astype(str)

classification_rep = classification_report(y_test_labels, y_pred_labels, target_names=class_labels)

print("Reporte de clasificación:")
print(classification_rep)

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test_labels, y_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicciones')
plt.ylabel('Etiquetas verdaderas')
plt.title('Matriz de Confusión')
plt.show()

### Representacion 2

In [ ]:
max_words = 20000
max_sequence_length = 100

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_categorical, test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(layers.Dense(80, activation='relu'))
model.add(layers.Dense(45, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 32
epochs = 10
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss en el conjunto de prueba: {loss}, Accuracy en el conjunto de prueba: {accuracy}')

In [ ]:
# Reporte de Clasificación
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

class_labels = label_encoder.classes_.astype(str)

classification_rep = classification_report(y_test_labels, y_pred_labels, target_names=class_labels)

print("Reporte de clasificación:")
print(classification_rep)

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test_labels, y_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicciones')
plt.ylabel('Etiquetas verdaderas')
plt.title('Matriz de Confusión')
plt.show()

### Representacion 3

In [ ]:
max_words = 10000
max_sequence_length = 100

In [ ]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X_sequences = tokenizer.texts_to_sequences(X)
X_padded = pad_sequences(X_sequences, maxlen=max_sequence_length)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_padded, y_categorical, test_size=0.2, random_state=42)

In [ ]:
embedding_dim = 100
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
# model.add(LSTM(128))
model.add(layers.Dense(50, activation='relu'))
# model.add(layers.Dropout(0.1))
model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
batch_size = 64
epochs = 15
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss en el conjunto de prueba: {loss}, Accuracy en el conjunto de prueba: {accuracy}')

In [ ]:
# Reporte de Clasificación
y_pred = model.predict(X_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

class_labels = label_encoder.classes_.astype(str)

classification_rep = classification_report(y_test_labels, y_pred_labels, target_names=class_labels)

print("Reporte de clasificación:")
print(classification_rep)

In [ ]:
# Matriz de confusión
cm = confusion_matrix(y_test_labels, y_pred_labels)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.xlabel('Predicciones')
plt.ylabel('Etiquetas verdaderas')
plt.title('Matriz de Confusión')
plt.show()